In [ ]:
import datetime
import os
import sys
import configparser
import pandas as pd
import networkx as nx
import operator as op
import util
import logging
import numpy as np
import networkx as nx

# **Constants**

In [ ]:

NONSENSE = "Nonsense_Mutation"
MISSENSE = "Missense_Mutation"
SPLICE ="Splice_Site"
FRAMESHIFT_DEL = "Frame_Shift_Del"
FRAMESHIFT_INS = "Frame_Shift_Ins"
INFRAME_DEL = "In_Frame_Del"
INFRAME_INS = "In_Frame_Ins"
_3UTR = "3'UTR"
_5UTR = "5'UTR"
NONSTOP_MUTATION = "Nonstop_Mutation"
TRANSLATION_START_SITE = "Translation_Start_Site"

In [ ]:
'''
Input: Pandas DataFrame with the MAF file information
OutPut: Binary Mutation Matrix (BMM), in a pandas DataFrame

Binary Mutation matrix format:
Rows are patients ((samples) and columns are genes.
1: gene is mutated in patient; 0: gene is not mutated in patient.

Example:
Hugo_Symbol g1  g2  g3  ... gm
p1  0   0   1   0
p2  1   0   0   1
p3  1   0   1   0
...
pn  1   0   0   0

'''

# **Buliding Binary Mutation Matrix (BMM)**

In [ ]:
def get_binary_mutation_matrix(maf):
    bmm = pd.crosstab(maf.Tumor_Sample_Barcode, maf.Hugo_Symbol).clip(upper=1)
    return bmm


'''
Input: Pandas DataFrame with the MAF file information
Output: Weighted Mutation Matrix (WMM), in a pandas DataFrame
'''
def get_weighted_mutation_matrix(maf, mutation_weights):
    patients = list(set(maf.Tumor_Sample_Barcode))
    genes = list(set(maf.Hugo_Symbol))
    mutation_types = list(set(maf.Variant_Classification))

    mutation_weights = {k: v for k, v in mutation_weights.items() if k in mutation_types}

    # cross tab to get the number of mutations of each pair gene-patient
    mut = pd.crosstab(maf.Hugo_Symbol, [maf.Tumor_Sample_Barcode, maf.Variant_Classification], values=maf.Variant_Classification, aggfunc=len, dropna=False)

    wmm = pd.DataFrame(index=patients, columns=genes) # Creating Weighted Mutation Matrix to keep the scores.
    wmm = wmm.fillna(0.0) # fill with 0s rather than NaNs

    # calcuting the scores for each pait gene-patient: s(p_i, g_j)
    for p in patients:
        mut[p] = mut[p].fillna(0) # fill with 0s rather NaNs
        for g in genes:
            mut_dict = mut[p].loc[g].to_dict() # getting dictionary of number of mutations for the pair patient-gene, organized by mutation type

            number_of_mutations = sum(mut_dict.values())
            sum_product = sum(mut_dict[k] * mutation_weights[k] for k in mut_dict)
            score = 0
            if number_of_mutations > 0:
                score =  round(sum_product / number_of_mutations, 3)
            wmm.at[p, g] = score
    return wmm

# **Building Weight Mutation Matrix (WMM)**

In [ ]:
'''
Input: Binary Mutation Matrix (BMM)
Output: An Alteration Matrix, in a dictionary

Each row contais patient (sample) in the first columns. The follow columns have the genes which are mutated in the sample.
Example:
p1  g1  g4  g13
p2  g7  g16 g53 g104
p3  g10 g12
...
pn  g1  g3
'''

In [ ]:
def get_alteration_matrix(bmm):
    matrix = dict()
    sample_list = bmm.index.tolist()
    for sample in sample_list:
        matrix[sample] = list(bmm.columns.values[bmm.loc[sample] == 1])

    return matrix

In [ ]:
def get_genes_from_bmm(bmm):
    return list(bmm)

'''
Input: Binary Mutation Matrix (BMM) and Weighted Mutation Matrix (WMM)
Output: A score for each gene
'''
def get_score_genes(bmm, wmm):
    samples = bmm.index.tolist()
    genes = list(bmm)
    gene_scores = dict()

    score_max = 0
    coeff = 0
    for g in genes:
        score = round(wmm[g].sum() / len(samples), 3) # getting the 'weighted frequence' of the scores in the gene
        gene_scores[g] = score
        if score > score_max: # keeping the biggest score
            score_max = score

    # putting scores between 0 and 1
    for g in gene_scores:
        gene_scores[g] = round(gene_scores[g] / score_max, 3)

    return gene_scores

# **Network Inputs**

In [ ]:
Input: the gene network file (reading using rb)
Output: a networkX graph
'''
def read_gene_network_nx(gene_network_file):
    mx = nx.read_edgelist(gene_network_file, delimiter='\t')
    return mx

'''"""
Input: a gene network (networkX), a threshold and a a list of genes considered in the analisys
Output: a Gene Strength Spreading Network (GSSN) and the biggest weight considering all the edges
'''""
from itertools import filterfalse
def create_gene_strength_spreading_network(gene_network_nx):
    gssn = nx.DiGraph()
    gene_network_nx.remove_edges_from(nx.selfloop_edges(gene_network_nx))
    import random
    gene_network_nx.edges = random.sample(gene_network_nx.edges, k=min(50000, len(gene_network_nx.edges)))
    max_weight = 0
    for g_i, g_j in gene_network_nx.edges:
        neighbors_i = dict(gene_network_nx[g_i])
        neighbors_j = dict(gene_network_nx[g_j])

        neighbors_j_out = {k:v for k,v in neighbors_j.items() if k not in neighbors_i}
        neighbors_j_out.pop(g_i, None)
        r_i = sum([neighbors_i[g]["weight"] for g in neighbors_i])
        r_j_out = sum([neighbors_j_out[g]["weight"] for g in neighbors_j_out])


        neighbors_i_out = {k:v for k,v in neighbors_i.items() if k not in neighbors_j}
        neighbors_i_out.pop(g_j, None)
        r_j = sum([neighbors_j[g]["weight"] for g in neighbors_j])
        r_i_out = sum([neighbors_i_out[g]["weight"] for g in neighbors_i_out])

        weight_ij = gene_network_nx[g_i][g_j]["weight"]
        s_ij = (1 + (r_i * r_j_out)) * weight_ij
        s_ji = (1 + (r_j * r_i_out)) * weight_ij

        gssn.add_edge(g_i, g_j, weight=s_ij)
        gssn.add_edge(g_j, g_i, weight=s_ji)

        if max(s_ij, s_ji) > max_weight:
            max_weight = max(s_ij, s_ji)
    return gssn, max_weight


# **Building Consensus Gene Network**

In [ ]:
def combine_gene_network(gene_network_nx_list):
    consensus_gene_network = nx.Graph()

    max_weight = 1
    for gene_network in gene_network_nx_list:
        for g_i, g_j in gene_network.edges:
            if not consensus_gene_network.has_edge(g_i, g_j):
                consensus_gene_network.add_edge(g_i, g_j, weight=1)
            else:
                consensus_gene_network[g_i][g_j]["weight"] += 1
                if consensus_gene_network[g_i][g_j]["weight"] > max_weight:
                    max_weight = consensus_gene_network[g_i][g_j]["weight"]

    for g_i, g_j in consensus_gene_network.edges:
        consensus_gene_network[g_i][g_j]["weight"] /= max_weight

    return consensus_gene_network

# **Gene Spreading Stregth and Neighbor Mutation Influence**

In [ ]:
'''
Input: gene scores, a directed and weighted gene network (networkX), the biggest weight considering all the edges of network
Output: a Gene Strength Spreading Network (GSSN) and the biggest weight considering all the edges
'''
def get_score_genes_neighbors(gene_scores, gssn, max_weight, consensus_gene_network):
    gene_scores_from_neighbors = {}

    genes_out = list(set(gssn) - set(gene_scores)) # genes of gssn not in gene_scores
    for g in genes_out:
        gene_scores[g] = 0
    max_score_neighbors = 0
    for g_i in gene_scores:
        score_from_neighbors = 0
        if gssn.has_node(g_i):
            g_neighbors = gssn.neighbors(g_i)
            for g_j in g_neighbors:
                w_ji = round(gssn[g_j][g_i]["weight"] / max_weight, 3)
                score_from_neighbors = score_from_neighbors + (gene_scores[g_j] * w_ji)
            if score_from_neighbors > max_score_neighbors:
                max_score_neighbors = score_from_neighbors
        gene_scores_from_neighbors[g_i] = round(score_from_neighbors, 3)
    for g_i in gene_scores_from_neighbors:
        gene_scores_from_neighbors[g_i] = round(gene_scores_from_neighbors[g_i] / max_score_neighbors, 3)
    return gene_scores_from_neighbors

# **Gene Mutations Score**

In [ ]:
def get_gene_scores(gene_scores_mutations, gene_scores_neighbors):
    gene_scores = {}

    for g in gene_scores_neighbors:
        scores = []
        scores.append(gene_scores_mutations[g])
        scores.append(gene_scores_neighbors[g])
        scores.append(gene_scores_mutations[g] + gene_scores_neighbors[g])
        gene_scores[g] = scores
    return gene_scores

In [ ]:
def main():
    input_parameters_file = sys.argv[1]
    print(input_parameters_file)
    cp = configparser.ConfigParser()
    cp.read(input_parameters_file)

    cp_input_type = cp["INPUT_TYPE"]
    input_type = int(cp_input_type["TYPE"])

    cp_input = cp["INPUT"]

    if input_type == 1:
        input_maf_file_name = cp_input["MAF_FILE_NAME"]
    elif input_type == 2:
        input_bmm_file_name = cp_input["BMM_FILE_NAME"]
        input_wmm_file_name = cp_input["WMM_FILE_NAME"]
    input_gene_network_files_name = cp_input["GENE_NETWORK_FILE_NAME"].split(" ")

    cp_mutation_weights = cp["VARIANT_CLASSIFICATION_WEIGHTS"]
    mutation_weights = {}
    mutation_weights["Nonsense_Mutation"] = float(cp_mutation_weights["NONSENSE"])
    mutation_weights["Missense_Mutation"] = float(cp_mutation_weights["MISSENSE"])
    mutation_weights["Splice_Site"] = float(cp_mutation_weights["SPLICE"])
    mutation_weights["Frame_Shift_Del"] = float(cp_mutation_weights["FRAMESHIFT_DEL"])
    mutation_weights["Frame_Shift_Ins"] = float(cp_mutation_weights["FRAMESHIFT_INS"])
    mutation_weights["In_Frame_Del"] = float(cp_mutation_weights["INFRAME_DEL"])
    mutation_weights["In_Frame_Ins"] = float(cp_mutation_weights["INFRAME_INS"])
    mutation_weights["3'UTR"] = float(cp_mutation_weights["_3UTR"])
    mutation_weights["5'UTR"] = float(cp_mutation_weights["_5UTR"])
    mutation_weights["Nonstop_Mutation"] = float(cp_mutation_weights["NONSTOP_MUTATION"])
    mutation_weights["Translation_Start_Site"] = float(cp_mutation_weights["TRANSLATION_START_SITE"])

    cp_output = cp["OUTPUT"]
    output_folder = cp_output["OUTPUT_FOLDER"]
    output_file = cp_output["OUTPUT_FILE"]
    output_wmm_file_name = output_file + ".wmm"
    output_bmm_file_name = output_file + ".bmm"
    output_am_file_name = output_file + ".am"
    output_gene_mutations_score = output_file + ".mutation.score"
    output_rgn_file_name = output_file + ".sgn"
    output_score_file_name = output_file + ".score"
    output_sample_report_file_name = output_file + ".sample.report"
    output_gene_report_file_name = output_file +  ".gene.report"
    output_network_report_files_name = []
    for gene_network_file_name in input_gene_network_files_name:
        output_network_report_files_name.append(output_file + '_' + gene_network_file_name + ".network.report")

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s', filename=output_folder + '/' + 'log.txt')
    logging.info('******** Starting ********')

    if input_type == 1:
        maf = pd.read_csv(input_maf_file_name, sep="\t", comment='#', usecols=["Hugo_Symbol", "Tumor_Sample_Barcode", "Variant_Classification"])
        print(maf.head())
        maf=maf.head(int(0.1*(len(maf))))
        wmm = get_weighted_mutation_matrix(maf, mutation_weights)
        bmm = get_binary_mutation_matrix(maf)
        util.create_txt_file_mutation_matrix(wmm, output_folder + "/" + output_wmm_file_name)
        util.create_txt_file_mutation_matrix(bmm, output_folder + "/" + output_bmm_file_name)
        am = get_alteration_matrix(bmm)
        util.get_sample_report(am, output_folder + "/" + output_sample_report_file_name)
        util.get_gene_report(bmm, output_folder + "/" + output_gene_report_file_name)
        mutated_genes = get_genes_from_bmm(bmm)
        logging.info('Input Type 1 - END matrix generations')

        gene_scores_mutations = get_score_genes(bmm, wmm)
        util.generate_gene_score_mutation_file(gene_scores_mutations, output_folder + "/" + output_gene_mutations_score)
        logging.info('END - getting gene_scores_mutations')
    elif input_type == 2:
        wmm = util.get_mutation_matrix_from_file(input_wmm_file_name)
        bmm = util.get_mutation_matrix_from_file(input_bmm_file_name)
        am = get_alteration_matrix(bmm)
        util.get_sample_report(am, output_folder + "/" + output_sample_report_file_name)
        util.get_gene_report(bmm, output_folder + "/" + output_gene_report_file_name)
        mutated_genes = get_genes_from_bmm(bmm)
        logging.info('Input Type 2 - END matrix generations')

        gene_scores_mutations = get_score_genes(bmm, wmm)
        util.generate_gene_score_mutation_file(gene_scores_mutations, output_folder + "/" + output_gene_mutations_score)
        logging.info('END - getting gene_scores_mutations')
    elif input_type == 3:
        gene_scores_mutations = util.get_mutation_score_from_file(output_folder + "/" + output_gene_mutations_score)
        mutated_genes = list(gene_scores_mutations)
    logging.info('END - matrix generations')

    gene_network_nx_list = []
    all_genes = set()

    for input_gene_network_file_name in input_gene_network_files_name:
        gene_network_file = open(input_gene_network_file_name, 'rb')
        gene_network_nx = read_gene_network_nx(gene_network_file)
        gene_network_nx_list.append(gene_network_nx)
        gene_network_file.close()

    consensus_gene_network = combine_gene_network(gene_network_nx_list)

    util.get_network_report(consensus_gene_network, output_folder + "/" + output_file + ".consensus.network.report")
    print("1")
    gssn, max_edge_weight = create_gene_strength_spreading_network(consensus_gene_network)

    consensus_gene_score_neighbors = get_score_genes_neighbors(gene_scores_mutations, gssn, max_edge_weight, consensus_gene_network)
    logging.info("END - " + input_gene_network_file_name +  "processing")


    logging.info("END - creating Gene Strength Spreading Networks (GSSNs)")
    logging.info("END - getting scores from neighbors")

    gene_scores = get_gene_scores(gene_scores_mutations, consensus_gene_score_neighbors)

    sorted_gene_scores = sorted(gene_scores.items(), key=lambda kv: kv[1][2], reverse=True)

    logging.info('END - sorting results')

    score_file = open(output_folder + "/" + output_score_file_name + ".mutatedGenes", "w")
    score_file.write("gene\tmutation_score\tscore_from_neighbors\tfinal_score\n")
    """"
    for g, scores in sorted_gene_scores:
        if g in mutated_genes:
            score_file.write("{}\t{:.3f}\t{:.3f}\t{:.3f}\n".format(g, scores[0], scores[1], scores[2]))
            #print("3")
    score_file.close()
    """
main()

# **Graph Based Prioritization with PageRank with Individualized Damping Factor (GBP-PR)**

In [ ]:
def page_rank_individualized_damping(consensus_gene_network, mu=0.1, num_iterations=100, alpha=0.85):
    num_genes = len(network)

    rank = np.ones(num_genes) / num_genes # Initialize variables
    lambda_values = np.zeros(num_genes)

    for gene in range(num_genes):
        ms_gj = np.sum(network[:, gene])  # In-degree of gene j
        lambda_values[gene] = ms_gj / (ms_gj + mu)

    for iteration in range(num_iterations):  # Perform PageRank with individualized damping factors
        new_rank = np.zeros(num_genes)

        for gene_i in range(num_genes):
            for gene_j in range(num_genes):
                if network[gene_i, gene_j] == 1:
                    ms_gi = np.sum(network[:, gene_i])  # In-degree of gene i
                    new_rank[gene_j] += (1 - lambda_values[gene_j]) * rank[gene_i]
                    new_rank[gene_j] += lambda_values[gene_j] * (rank[gene_i] / ms_gi)

        new_rank = (1 - alpha) + alpha * new_rank # Apply the damping factor

        new_rank /= sum(new_rank) # Normalize the rank to sum to 1

    if np.allclose(new_rank, rank, atol=1e-6): # Check for convergence


        rank = new_rank

    return rank


gene_network_nx_list = []
all_genes = set()

"""
for input_gene_network_file_name in input_gene_network_files_name:
        gene_network_file = open(input_gene_network_file_name, 'rb')
        gene_network_nx = read_gene_network_nx(gene_network_file)
        gene_network_nx_list.append(gene_network_nx)
        gene_network_file.close()
 """
 onsensus_gene_network = gene_network(gene_network_nx_list)

    # Replace this with your mu value (individualized damping factor)
mu_value = 0.1
ranks = page_rank_individualized_damping(consensus_gene_network, mu=mu_value)


ranked_genes = np.argsort(ranks)[::-1]
print("Ranked genes:", ranked_genes)

# **Graph Based Prioritization with Standard PageRank**

In [ ]:
def pageRank(consensus_gene_network, damping_factor=0.85, max_iterations=100, epsilon=1e-6):
    num_nodes = len(consensus_gene_network)
    initial_score = np.ones(num_nodes) / num_nodes  # Initialize scores equally for all nodes
    transition_matrix = build_transition_matrix(consensus_gene_network, num_nodes, damping_factor)

    # Iterate to calculate PageRank scores
    prev_scores = np.copy(initial_score)
    for _ in range(max_iterations):
        new_scores = (1 - damping_factor) / num_nodes + damping_factor * np.dot(transition_matrix, prev_scores)

        # Check for convergence
        if np.linalg.norm(new_scores - prev_scores, 2) < epsilon:
            return new_scores

        prev_scores = new_scores

    return prev_scores

def build_transition_matrix(consensus_gene_network, num_nodes, damping_factor):
    transition_matrix = np.zeros((num_nodes, num_nodes))

    for i in range(num_nodes):
        num_outlinks = sum(consensus_gene_network[i])
        if num_outlinks == 0:
            # Handle nodes with no outlinks by assuming equal probability to all nodes
            transition_matrix[i] = np.ones(num_nodes) / num_nodes
        else:
            for j in range(num_nodes):
                transition_matrix[i, j] = damping_factor * (consensus_gene_network[j, i] / num_outlinks)
                transition_matrix[i, j] += (1 - damping_factor) / num_nodes

    return transition_matrix
"""
def combine_gene_network(gene_network_nx_list):
    consensus_gene_network = nx.Graph()

    max_weight = 1
    for gene_network in gene_network_nx_list:
        for g_i, g_j in gene_network.edges:
            if not consensus_gene_network.has_edge(g_i, g_j):
                consensus_gene_network.add_edge(g_i, g_j, weight=1)
            else:
                consensus_gene_network[g_i][g_j]["weight"] += 1
                if consensus_gene_network[g_i][g_j]["weight"] > max_weight:
                    max_weight = consensus_gene_network[g_i][g_j]["weight"]

    for g_i, g_j in consensus_gene_network.edges:
        consensus_gene_network[g_i][g_j]["weight"] /= max_weight

    return consensus_gene_network
"""

consensus_gene_network = combine_gene_network(gene_network_nx_list)
# Run PageRank on the gene network
scores = pageRank(consensus_gene_network)

# Display the PageRank scores for each gene
for i, score in enumerate(scores):
    print(f"Gene {i + 1}: PageRank Score = {score:.4f}")


In [ ]:
def personalized_pagerank(consensus_gene_network, gene_factors, mu, num_iterations=100, alpha=0.85):
    num_genes = len(network)

    # Initialize PageRank scores
    pagerank = np.ones(num_genes) / num_genes

    for iteration in range(num_iterations):
        new_pagerank = np.zeros(num_genes)

        for j in range(num_genes):
            lambda_j = gene_factors[j] / (gene_factors[j] + mu)

            # Calculate the personalized PageRank for gene j
            new_pagerank[j] = (1 - lambda_j) * alpha + lambda_j * sum(
                pagerank[i] / np.sum(network[i]) for i in range(num_genes) if network[i][j] == 1
            )

        # Normalize the PageRank scores
        new_pagerank /= np.sum(new_pagerank)

        # Check for convergence
        if np.allclose(new_pagerank, pagerank, atol=1e-6):
            break

        pagerank = new_pagerank

    return pagerank

    # Replace this with your damping factor mu
    mu = 0.1
    pageranks = personalized_pagerank(consensus_gene_network, gene_factors, mu)

    # Print the PageRank scores for each gene
    for gene, score in enumerate(pageranks):
        print(f"Gene {gene}: {score}")


# **Graph Based Prioritization with Colley (GBP-Colley)**

In [ ]:
import numpy as np

def colley_ranking(adjacency_matrix, consensus_gene_network):
    # Number of genes in the network
    num_genes = len(adjacency_matrix)

    # Create a Colley matrix
    colley_matrix = np.zeros((num_genes, num_genes))

    # Populate the Colley matrix based on interactions
    for i in range(num_genes):
        for j in range(num_genes):
            if i == j:
                colley_matrix[i][j] = 0.5  # Diagonal elements
            else:
                wins = 0
                losses = 0

                # Count wins and losses based on interactions
                for k in range(num_genes):
                    if adjacency_matrix[i][k] == 1 and adjacency_matrix[j][k] == 0:
                        wins += 1
                    elif adjacency_matrix[i][k] == 0 and adjacency_matrix[j][k] == 1:
                        losses += 1

                # Update the Colley matrix
                colley_matrix[i][j] = 0.5 + wins
                colley_matrix[j][i] = 0.5 + losses

    # Compute the Colley rankings
    colley_rankings = np.dot(np.linalg.inv(colley_matrix), gene_data)

    # Sort the genes based on their Colley ranking
    ranked_genes = [(gene, ranking) for gene, ranking in enumerate(colley_rankings)]
    ranked_genes.sort(key=lambda x: -x[1])  # Sort in descending order

    return ranked_genes

    print("Colley Rankings:")
    for rank, (gene, ranking) in enumerate(ranked_genes, start=1):
        print(f"Rank {rank}: Gene {gene}, Ranking = {ranking}")


# **Graph Based Prioritization with massey (GBP-Massey)**

In [ ]:
def massey_ranking(data, consensus_gene_network):
    # Determine the number of genes
    num_genes = len(consensus_gene_network)

    # Initialize the Massey matrix and the b vector
    massey_matrix = np.zeros((num_genes, num_genes))
    b = np.zeros(num_genes)

    # Populate the Massey matrix and b vector
    for i in range(num_genes):
        for j in range(num_genes):
            if i != j:
                # Count the number of matches and mismatches between gene i and gene j
                matches = sum([data[gene_i][gene_j] == 1 for (gene_i, gene_j) in consensus_gene_network])
                mismatches = sum([data[gene_i][gene_j] == -1 for (gene_i, gene_j) in consensus_gene_network])

                # Update the Massey matrix and b vector
                massey_matrix[i][j] = -(matches - mismatches)
                massey_matrix[i][i] += matches + mismatches
                b[i] += 0.5 * (matches - mismatches)

    # Solve the Massey system of equations
    massey_ratings = np.linalg.solve(massey_matrix, b)

    # Create a list of (gene, rating) tuples
    gene_ratings = [(gene, rating) for gene, rating in enumerate(massey_ratings)]

    # Sort genes by their ratings in descending order
    gene_ratings.sort(key=lambda x: -x[1])

    # Return the ranked genes
    ranked_genes = [gene for gene, _ in gene_ratings]
    return ranked_genes


    ranked_genes = massey_ranking(data, consensus_gene_network)

    # Print the ranked genes
    print("Ranked Genes:", ranked_genes)

  """
  class MasseyRanking:
    def __init__(self, gene_interactions):
        self.gene_interactions = gene_interactions
        self.gene_count = len(gene_interactions)
        self.match_results = []

    def add_match_result(self, result_vector):
        if len(result_vector) == self.gene_count:
            self.match_results.append(result_vector)
        else:
            raise ValueError("Result vector length should match the number of genes in the network.")

    def calculate_massey_ratings(self):
        wins = [0] * self.gene_count
        losses = [0] * self.gene_count

        for result in self.match_results:
            for i in range(self.gene_count):
                for j in range(i + 1, self.gene_count):
                    if result[i] > result[j]:
                        wins[i] += 1
                        losses[j] += 1
                    elif result[i] < result[j]:
                        wins[j] += 1
                        losses[i] += 1

        massey_ratings = []
        for i in range(self.gene_count):
            rating = wins[i] - losses[i]
            massey_ratings.append((i, rating))

        massey_ratings.sort(key=lambda x: x[1], reverse=True)

        return massey_ratings

print(f"{rank}. {gene_interactions[gene_index]}: {rating}")
"""

# **Graph Based Prioritization with Keener (GBP-Keener)**

In [ ]:
import networkx as nx

def keener_ranking(consensus_gene_network):
    # Create a directed graph to represent the consensus gene interaction network
    G = nx.DiGraph(consensus_gene_network)

    # Initialize ratings for all genes to 1
    ratings = {gene: 1 for gene in G.nodes()}

    # Maximum number of iterations for convergence
    max_iterations = 1000

    # Convergence threshold for ratings
    convergence_threshold = 1e-6

    # Iteratively update ratings until convergence or maximum iterations reached
    for iteration in range(max_iterations):
        new_ratings = {}

        # Calculate the sum of ratings from neighboring genes for each gene
        for gene in G.nodes():
            neighbor_ratings_sum = sum(ratings[neighbor] for neighbor in G.predecessors(gene))
            new_ratings[gene] = neighbor_ratings_sum

        # Normalize the ratings to ensure they sum to 1
        total_rating = sum(new_ratings.values())
        for gene in new_ratings:
            new_ratings[gene] /= total_rating

        # Check for convergence
        convergence = True
        for gene in G.nodes():
            if abs(new_ratings[gene] - ratings[gene]) > convergence_threshold:
                convergence = False
                break

        if convergence:
            break
        else:
            ratings = new_ratings

    # Sort genes by their final ratings
    ranked_genes = sorted(ratings.keys(), key=lambda x: ratings[x], reverse=True)

    return ranked_genes

consensus_network = nx.DiGraph()

# Get the ranked genes using Keener method
ranked_genes = keener_ranking(consensus_network)

# Print the ranked genes
print("Ranked Genes:")
for i, gene in enumerate(ranked_genes):
    print(f"{i+1}. {gene}")
